In [1]:
import numpy as np
import shlex
import subprocess
import sys
import pprint

In [2]:
def get_docsim(fn, docno2sim = {}):
    p = open(fn)
    docno_list = []
    sim_list = []
    count = 0
    for l in p:
        ls = l[:-1].split()
        if len(ls) == 7:
            qid, iternum, docno, rank, sim, run_id, label = ls
        else:
            qid, iternum, docno, rank, sim, run_id = ls
        if docno+"_"+qid in docno2sim:
            # print("docno {} already in docno2sim".format(docno))
            count += 1
        else:
            docno2sim[docno+"_"+qid] = float(sim)
        sim_list.append(float(sim))
        docno_list.append(docno+"_"+qid)
    maxSim = max(sim_list)
    minSim = min(sim_list)
    # print("count: {}".format(count))
    for docnoqid in docno_list:
        docno2sim[docnoqid] = (docno2sim[docnoqid] - minSim) / (maxSim - minSim)
    return docno2sim

In [3]:
train2year = {'train_2011': '2011', 'test_2011': '2012', 'train_2013': '2013', 'test_2013': '2014'}
docno2sim = {}
docno2sim_ql = {}
for train in train2year:
    if train == "test_2013": continue
    print(train)
    year = train2year[train]
    fn = "baseline/run.microblog{}.QL.txt".format(year)
    docno2sim_ql = get_docsim(fn, docno2sim_ql)
    print(len(docno2sim_ql))

train_2013
46192
test_2011
96071
train_2011
136765


In [14]:
def get_map_inter(docno2sim, docno2sim_ql, l, debug=False, mode="train"):
    docno2sim_inter = {}
    for docnoqid in docno2sim:
        if docnoqid in docno2sim_ql:
            docno2sim_inter[docnoqid] = docno2sim[docnoqid] * l + docno2sim_ql[docnoqid] * (1 -l)
    if mode == "train":
        fn_inter = "predict.inter.{}.{}.l{:.2f}.{}".format(mode, model, l, testset)
    else:
        fn_inter = "predict.inter.{}.{}.{}".format(mode, model, testset)
    f_inter = open(fn_inter, "w")
    if debug:
        print("total pairs: {}, written to file: {}".format(len(docno2sim_inter), fn_inter))
    for docnoqid in docno2sim_inter:
        temp = docnoqid.split("_")
        docno = temp[0]
        qid = temp[1]
        score = docno2sim_inter[docno+"_"+qid]
        f_inter.write('{} 0 {} 0 {} {}\n'.format(qid, docno, score, model))
    # always remember to close the file !
    f_inter.close()
    
    fn_qrels = "qrels_all.txt"
    cmd = "/bin/sh run_eval.sh {} {}".format(fn_inter, fn_qrels)
    pargs = shlex.split(cmd)
    p = subprocess.Popen(pargs, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    pout, perr = p.communicate()
    if debug:
        print("running {}".format(cmd))
        if len(pout) != 0:
            print(pout.decode('utf-8'))
        else:
            print(perr.decode('utf-8'))
    if sys.version_info[0] < 3:
        lines = pout.split('\n')
    else:
        lines = pout.split(b'\n')
    Map = float(lines[0].strip().split()[-1])
    Mrr = float(lines[1].strip().split()[-1])
    P30 = float(lines[2].strip().split()[-1])
    return Map, P30

def get_inter_tune(testset, model, l, debug=False):
    train_set = set(['train_2011', 'test_2011', 'train_2013', 'test_2013'])
    train_set.remove(testset)
    train2year = {'train_2011': '2011', 'test_2011': '2012', 'train_2013': '2013', 'test_2013': '2014'}
    docno2sim = {}
    docno2sim_ql = {}
    for train in train_set:
        year = train2year[train]
        fn = "baseline/run.microblog{}.QL.txt".format(year)
        docno2sim_ql = get_docsim(fn, docno2sim_ql)
        
    fn = "predict.train.{}.{}.txt".format(model, testset)
    docno2sim = get_docsim(fn, docno2sim)
    return get_map_inter(docno2sim, docno2sim_ql, l, debug=debug)

def select_l(testset, model, debug=False):
    maxL = 0
    maxMap = 0
    ll = []
    for l in range(0, 100, 5):
        l = l / 100.0
        # print("trying lambda: {:.2f}".format(l))
        Map, P30 = get_inter_tune(testset, model, l, debug=debug)
        ll.append(Map)
        if Map > maxMap:
            maxMap = Map
            maxL = l
    # print("best lambda: {} with MAP: {}".format(maxL, maxMap))
    return maxL, ll

def get_inter_test(testset, model, l):
    train2year = {'train_2011': '2011', 'test_2011': '2012', 'train_2013': '2013', 'test_2013': '2014'}
    docno2sim = {}
    docno2sim_ql = {}
    year = train2year[testset]
    fn = "baseline/run.microblog{}.QL.txt".format(year)
    docno2sim_ql = get_docsim(fn, docno2sim_ql)
    fn = "predict.test.{}.{}.txt".format(model, testset)
    docno2sim = get_docsim(fn, docno2sim)
    return get_map_inter(docno2sim, docno2sim_ql, l, mode="test")

In [18]:
# for l in range(0, 101, 5):
#     l = l / 100.0
# testset = "test_2013"
# model = "duet"
trains = ['train_2011', 'test_2011', 'train_2013', 'test_2013']
models = ["dssm", "drmm", "cdssm", "matchpyramid", "knrm", "duet"]
for testset in trains:
    for model in models:
        maxL, ll = select_l(testset=testset, model=model, debug=False)
        testMap, P30 = get_inter_test(testset=testset, model=model, l=maxL)
        print("Model: {}, testset: {}, Map = {:.4f}, P30 = {:.4f} with lambda = {}".format(model, testset, testMap, P30, maxL))

Model: dssm, testset: train_2011, Map = 0.3589, P30 = 0.4143 with lambda = 0.15
Model: drmm, testset: train_2011, Map = 0.3477, P30 = 0.4034 with lambda = 0.15
Model: cdssm, testset: train_2011, Map = 0.3380, P30 = 0.3952 with lambda = 0.15
Model: matchpyramid, testset: train_2011, Map = 0.3707, P30 = 0.4190 with lambda = 0.15
Model: knrm, testset: train_2011, Map = 0.3576, P30 = 0.4000 with lambda = 0.0
Model: duet, testset: train_2011, Map = 0.3576, P30 = 0.4000 with lambda = 0.0
Model: dssm, testset: test_2011, Map = 0.1777, P30 = 0.2989 with lambda = 0.55
Model: drmm, testset: test_2011, Map = 0.2213, P30 = 0.3537 with lambda = 0.35
Model: cdssm, testset: test_2011, Map = 0.2091, P30 = 0.3311 with lambda = 0.0
Model: matchpyramid, testset: test_2011, Map = 0.2170, P30 = 0.3362 with lambda = 0.25
Model: knrm, testset: test_2011, Map = 0.2277, P30 = 0.3520 with lambda = 0.05
Model: duet, testset: test_2011, Map = 0.2243, P30 = 0.3644 with lambda = 0.3
Model: dssm, testset: train_2013

In [ ]:
# lam = 0
# for l in ll:
#     print("lambda = {:.2f},    map_interpolation = {}".format(lam, l))
#     lam += 0.05
